# Summary

The methods in this notebook accept either an excel file or a text representation of a numpy array as a data source

They enable the user to: 


*   display mazes in the command line using ascii symbols
*   transform the basic mazes into a graph of the distances between each "tile"

*   find the fastest path from a single source to a single target using Dijkstra’s algorithm and print the solution to the command line
*   display the found solution on top of the ascii output of the maze


This code was originally written for a 48 hour take home exam


# Setup

In [50]:
import pandas as pd
import numpy as np
from scipy.sparse.csgraph import shortest_path


#This needs to be set up individually and cannot be tested directly
#I'm saving files created in excel in my drive to quickly create and test new mazes
#Link to tutorial: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92
#However all test mazes have been integrated into the jupyter file

#from google.colab import drive 
#drive.mount('/content/drive')


# Parsing and graphically verifying the mazes

Excel Parser

In [51]:
#Parses Excel Files and returns a numpy string array, this sets na values to ' ' for ease of printing
def importMaze(loc_str):
  pd_df = pd.read_excel(loc_str, index_col=None, header=None)
  pd_df=pd_df.fillna(" ")
  np_array = pd_df.to_numpy(dtype=str)
  return np_array

Maze Renderer

In [52]:
#Renders a maze, expects a 2 dimensional np array
def printMaze(np_str_array):
  print('')#To secure a new line to avoid graphical artifacts
  for i in range(np_str_array.shape[1] + 2): #renders the top line
    print('-',end='')
    if(i==np_str_array.shape[1]+1):print(' ')

  for i in range(np_str_array.shape[0]):
    for j in range(np_str_array.shape[1]):
      if(j==0):print('|',end='')
      print(np_str_array[i,j],end='')
      if(j==np_str_array.shape[1]-1): print('|')

  for i in range(np_str_array.shape[1] + 2): #renders the bottom line
    print('-',end='')   

Importing test data

# In here lies the test data as text, shield your eyes 😏

In [53]:
####This is how I originally imported the test data
#All outputs were written to the console and saved as text for future ease of use
#and to allow direct testing by different users

#test_case0 = importMaze('/content/drive/MyDrive/Colab Notebooks/data/testmaze0.xlsx')

#This allows me to output arrays to the console without any abbreviation, but 
#might cause issues on some system, rerunning the output seems to fix this in colab:

#import sys
#np.set_printoptions(threshold=sys.maxsize) 

test_case = np.array([['#', ' ', ' ', ' ', '#', 'T'],
       ['#', ' ', '#', ' ', '#', ' '],
       ['#', ' ', '#', ' ', '#', ' '],
       ['#', ' ', '#', ' ', '#', ' '],
       ['#', ' ', '#', ' ', '#', ' '],
       ['#', ' ', '#', ' ', '#', ' '],
       [' ', ' ', '#', ' ', '#', ' '],
       [' ', ' ', '#', ' ', '#', ' '],
       [' ', ' ', '#', ' ', '#', ' '],
       ['S', ' ', '#', ' ', ' ', ' ']])

test_case0 = np.array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', 'T'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        '#', ' ', ' ', ' ', ' ', '#', '#', '#', '#', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' '],
       ['S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ']])


test_case1 = np.array([[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', 'T'],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', '#', '#', '#', '#', '#', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['S', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']])

test_case2 = np.array([['S', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', '#', '#', '#', '#', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', '#', '#', '#', '#', '#', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       ['#', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', 'T'],
       [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       ['#', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ', ' ', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       [' ', ' ', '#', ' ', ' ', ' ', '#', '#', '#', '#', '#', ' ', '#',
        '#', '#', '#', '#', ' ', ' ', ' ', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       ['#', '#', '#', '#', '#', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '#',
        ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ',
        ' ', ' '],
       ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#',
        '#', '#', '#', '#', '#', '#', '#', '#', ' ', ' ', ' ', ' ', ' ',
        ' ', ' ']])

Graph "Generator"

# Workhorse methods

In [54]:
####The purpose of this function is to generate a graph of the original maze
def generateGraph(a_maze):
  #initialize a graph 
  dim_maze = a_maze.shape[0] * a_maze.shape[1]
  maze_graph = np.full((dim_maze,dim_maze),0)
  #Please ignore the large amount of test outputs, 
  #the last time I did any actual programming was in the Java 1.6 era:)
  
  #iterate over all elements and check if a path to the neighbor to the right or
  #down exists if feasible:

  for i in range(a_maze.shape[0]): 
    
      for j in range(a_maze.shape[1]):


        if(i<a_maze.shape[0] - 1):

          if(j<a_maze.shape[1] - 1): #behavior for all but the last column
            #print(str(i)+" "+str(j), a_maze[i,j])
            if(a_maze[i,j] != '#'): 
              if(a_maze[i,j + 1] != '#'): #Look to the right of the current postion
              
                maze_graph[i * a_maze.shape[1] + j,i * a_maze.shape[1] + j +1]=1
                maze_graph[i * a_maze.shape[1] + j + 1,i * a_maze.shape[1] + j]=1

              if(a_maze[i + 1,j]!='#'): #Look down from the current postion
                              
                maze_graph[i * a_maze.shape[1] + j,(i + 1) * a_maze.shape[1] + j]=1
                maze_graph[(i + 1) * a_maze.shape[1] + j,i * a_maze.shape[1] + j]=1
          else:
            if(a_maze[i,j]!='#'): 
                if(a_maze[i + 1,j]!='#'): #Look down from the current postion
                  maze_graph[i * a_maze.shape[1] + j,(i + 1) * a_maze.shape[1] + j]=1
                  maze_graph[(i + 1) * a_maze.shape[1] + j,i * a_maze.shape[1] + j]=1
        else:

          if(j<a_maze.shape[1] - 1): #behavior for all but the last column
            #print(str(i)+" "+str(j), a_maze[i,j])
            if(a_maze[i,j]!='#'): 
              if(a_maze[i,j + 1] != '#'): #Look to the right of the current postion
              
                maze_graph[i * a_maze.shape[1] + j,i * a_maze.shape[1] + j +1]=1
                maze_graph[i * a_maze.shape[1] + j + 1,i * a_maze.shape[1] + j]=1

  return maze_graph

Calculate the fastest route

In [55]:
###This is taken wholesale from https://stackoverflow.com/questions/53074947/examples-for-search-graph-using-scipy/53078901
###I take no credit for this implementation
#Reconstructs the shortest path via precedessors
def get_path(Pr, i, j):
    path = [j]
    k = j
    while Pr[i, k] != -9999:
        path.append(Pr[i, k])
        k = Pr[i, k]
    return path[::-1]


Putting it all together

In [56]:
def mazeSolver(a_maze):
  maze_graph = generateGraph(a_maze)
  #Takes a graph in the form of a numpy array
  #and returns not only the shortest distances, 
  #but allows via predecessors to reconstruct the shortest path
  #See method get_path, a link to the original implementation can be found there
  #Dijkstra’s algorithm was selected out of familiarity
  D, Pr = shortest_path(maze_graph, directed=False, method='D', return_predecessors=True)
  #Detects source and target, assumes only a single instance of each:
  source = np.where(a_maze=='S')
  target = np.where(a_maze=='T')
  a_path = get_path(Pr,source[0][0] * a_maze.shape[1] + source[1][0], target[0][0] * a_maze.shape[1] + target[1][0])
  #iterates over all nodes on the shortest path and translates them to the original coordinate system:
  for i in range(0,len(a_path)):#Crude but allows me to detect when I'm at the last iteration
    if(i<len(a_path) - 1):
      print('(' +str(a_path[i]//a_maze.shape[1])+", " +str(a_path[i]%a_maze.shape[1])+') -> ', end='')
    else:
      print('(' +str(a_path[i]//a_maze.shape[1])+", " +str(a_path[i]%a_maze.shape[1])+')')

Solution Renderer

In [57]:
#For a more detailed explanation see maze_graph
def printSolution(a_maze):
  maze_graph = generateGraph(a_maze)
  D, Pr = shortest_path(maze_graph, directed=False, method='D', return_predecessors=True)
  source = np.where(a_maze=='S')
 
  target = np.where(a_maze=='T')
  a_path = get_path(Pr,source[0][0]*a_maze.shape[1]+source[1][0],target[0][0]*a_maze.shape[1]+target[1][0])
  b_maze = np.copy(a_maze) #This is to avoid overwriting the original array
  for i in range(1,len(a_path) - 1):
    b_maze[(a_path[i] // a_maze.shape[1]),(a_path[i] % a_maze.shape[1])]=':'

  printMaze(b_maze)  

# Unit tests

In [58]:
mazeSolver(test_case0)
printMaze(test_case0)
printSolution(test_case0)

(24, 0) -> (24, 1) -> (24, 2) -> (24, 3) -> (24, 4) -> (24, 5) -> (24, 6) -> (24, 7) -> (24, 8) -> (24, 9) -> (24, 10) -> (24, 11) -> (24, 12) -> (23, 12) -> (22, 12) -> (21, 12) -> (20, 12) -> (20, 11) -> (20, 10) -> (20, 9) -> (20, 8) -> (20, 7) -> (19, 7) -> (19, 6) -> (18, 6) -> (18, 5) -> (17, 5) -> (17, 4) -> (16, 4) -> (15, 4) -> (14, 4) -> (13, 4) -> (12, 4) -> (12, 5) -> (12, 6) -> (12, 7) -> (12, 8) -> (12, 9) -> (12, 10) -> (12, 11) -> (12, 12) -> (12, 13) -> (12, 14) -> (12, 15) -> (12, 16) -> (12, 17) -> (12, 18) -> (12, 19) -> (11, 19) -> (10, 19) -> (10, 20) -> (9, 20) -> (9, 21) -> (8, 21) -> (7, 21) -> (7, 20) -> (7, 19) -> (7, 18) -> (7, 17) -> (7, 16) -> (7, 15) -> (7, 14) -> (7, 13) -> (7, 12) -> (7, 11) -> (7, 10) -> (7, 9) -> (7, 8) -> (7, 7) -> (7, 6) -> (7, 5) -> (6, 5) -> (5, 5) -> (5, 4) -> (4, 4) -> (4, 3) -> (3, 3) -> (2, 3) -> (2, 4) -> (2, 5) -> (2, 6) -> (2, 7) -> (2, 8) -> (2, 9) -> (2, 10) -> (2, 11) -> (2, 12) -> (2, 13) -> (2, 14) -> (2, 15) -> (2, 16

In [59]:
mazeSolver(test_case1)
printMaze(test_case1)
printSolution(test_case1)

(25, 0) -> (24, 0) -> (23, 0) -> (23, 1) -> (22, 1) -> (21, 1) -> (20, 1) -> (19, 1) -> (18, 1) -> (17, 1) -> (16, 1) -> (15, 1) -> (14, 1) -> (13, 1) -> (12, 1) -> (11, 1) -> (10, 1) -> (9, 1) -> (8, 1) -> (7, 1) -> (6, 1) -> (6, 2) -> (6, 3) -> (7, 3) -> (8, 3) -> (9, 3) -> (10, 3) -> (11, 3) -> (12, 3) -> (13, 3) -> (14, 3) -> (15, 3) -> (16, 3) -> (17, 3) -> (18, 3) -> (19, 3) -> (20, 3) -> (21, 3) -> (22, 3) -> (23, 3) -> (24, 3) -> (25, 3) -> (25, 4) -> (25, 5) -> (26, 5) -> (26, 6) -> (26, 7) -> (26, 8) -> (26, 9) -> (26, 10) -> (26, 11) -> (25, 11) -> (24, 11) -> (23, 11) -> (22, 11) -> (21, 11) -> (20, 11) -> (19, 11) -> (18, 11) -> (17, 11) -> (16, 11) -> (15, 11) -> (14, 11) -> (13, 11) -> (12, 11) -> (11, 11) -> (10, 11) -> (9, 11) -> (8, 11) -> (7, 11) -> (6, 11) -> (5, 11) -> (4, 11) -> (3, 11) -> (3, 12) -> (3, 13) -> (3, 14) -> (3, 15) -> (3, 16) -> (3, 17) -> (4, 17) -> (5, 17) -> (6, 17) -> (7, 17) -> (8, 17) -> (9, 17) -> (10, 17) -> (11, 17) -> (12, 17) -> (13, 17) 

In [60]:
mazeSolver(test_case2)
printMaze(test_case2)
printSolution(test_case2)

(0, 0) -> (0, 1) -> (0, 2) -> (0, 3) -> (0, 4) -> (0, 5) -> (0, 6) -> (0, 7) -> (0, 8) -> (0, 9) -> (0, 10) -> (0, 11) -> (0, 12) -> (0, 13) -> (0, 14) -> (0, 15) -> (0, 16) -> (0, 17) -> (0, 18) -> (0, 19) -> (0, 20) -> (0, 21) -> (0, 22) -> (0, 23) -> (1, 23) -> (2, 23) -> (3, 23) -> (4, 23) -> (5, 23) -> (6, 23) -> (6, 24) -> (7, 24) -> (8, 24) -> (8, 25) -> (9, 25) -> (10, 25) -> (10, 26) -> (10, 27) -> (11, 27) -> (12, 27)

------------------------------ 
|S                           |
|                            |
| ######################     |
|      #             #       |
| #### ##### ##### #####     |
|  #   #     #       #       |
|      #     #       #       |
|  #   #     #       #       |
|  #   #     #       #       |
|# #   #     #       #       |
|  #   #     #       #       |
|  #   #     #       #       |
|  #   #     #       #      T|
|  #   ##### #####   #       |
|  #   #         #   #       |
|  #   #         #   #       |
|  #   #         #   #       |
|  #   #